In [2]:
import pandas as pd

from src.main import main

In [3]:
df = main("data junior")

In [6]:
len(df)

140

In [5]:
pd.set_option('display.max_colwidth', None)
df.head()

,titles,companyNames,jobSnippets,locations,dates,clean_dates,links
23,Junior Business Analyst intern,Jonar,"As a Junior Business Analyst, you’ll learn how to analyze and work with complex data, design workflows and use cases, and provide suggestions on how to improve…","Montréal, QC",Today,0,https://ca.indeed.com/jobs?q=Junior%20Business%20Analyst%20intern%20Jonar
82,Data Scientist I,TD Bank,Capture and analyze information or data on current and future trends. You will employ these disciplines to help create data related solutions to drive business…,"Toronto, ON",Today,0,https://ca.indeed.com/jobs?q=Data%20Scientist%20I%20TD%20Bank
17,Junior Machine Learning Scientist,Ecoation,"Analyzing customer horticultural data to provide meaningful insights. Occasional field visits to greenhouses to collect data, develop or test approaches.","North Vancouver, BC",Today,0,https://ca.indeed.com/jobs?q=Junior%20Machine%20Learning%20Scientist%20Ecoation
60,SAS Developer,MSi Corp (Bell Canada),Document data mapping diagrams and data dictionaries. Work with business analysts to understand the business requirements for data modelling and data context.,"Ottawa, ON",3 days ago,3,https://ca.indeed.com/jobs?q=SAS%20Developer%20MSi%20Corp%20%28Bell%20Canada%29
69,Data Engineer - Level I,Swift Medical,"As the world leader in digital wound care management, we deliver advanced wound care visualization and touchless 3D measurement through our smartphone-ready…","Toronto, ON",3 days ago,3,https://ca.indeed.com/jobs?q=Data%20Engineer%20-%20Level%20I%20Swift%20Medical


In [8]:
# this output is best viewed in VSCode
df

,titles,companyNames,jobSnippets,locations,dates,clean_dates,links
23,Junior Business Analyst intern,Jonar,"As a Junior Business Analyst, you’ll learn how to analyze and work with complex data, design workflows and use cases, and provide suggestions on how to improve…","Montréal, QC",Today,0,https://ca.indeed.com/jobs?q=Junior%20Business%20Analyst%20intern%20Jonar
82,Data Scientist I,TD Bank,Capture and analyze information or data on current and future trends. You will employ these disciplines to help create data related solutions to drive business…,"Toronto, ON",Today,0,https://ca.indeed.com/jobs?q=Data%20Scientist%20I%20TD%20Bank
17,Junior Machine Learning Scientist,Ecoation,"Analyzing customer horticultural data to provide meaningful insights. Occasional field visits to greenhouses to collect data, develop or test approaches.","North Vancouver, BC",Today,0,https://ca.indeed.com/jobs?q=Junior%20Machine%20Learning%20Scientist%20Ecoation
60,SAS Developer,MSi Corp (Bell Canada),Document data mapping diagrams and data dictionaries. Work with business analysts to understand the business requirements for data modelling and data context.,"Ottawa, ON",3 days ago,3,https://ca.indeed.com/jobs?q=SAS%20Developer%20MSi%20Corp%20%28Bell%20Canada%29
69,Data Engineer - Level I,Swift Medical,"As the world leader in digital wound care management, we deliver advanced wound care visualization and touchless 3D measurement through our smartphone-ready…","Toronto, ON",3 days ago,3,https://ca.indeed.com/jobs?q=Data%20Engineer%20-%20Level%20I%20Swift%20Medical
71,Engineering Analyst I,TES - The Employment Solution,"Knowledge of statistical, mathematical and data analysis skills. Technically inclined with ability to understand and solve problems by formulating conclusions…","Markham, ON",3 days ago,3,https://ca.indeed.com/jobs?q=Engineering%20Analyst%20I%20TES%20-%20The%20Employment%20Solution
75,Jr. Data Analyst,Flex Surveys,Translating complex data and research outcomes into engaging and actionable formats for clients. We offer an array of survey consulting services and develop…,"Greater Toronto Area, ON",3 days ago,3,https://ca.indeed.com/jobs?q=Jr.%20Data%20Analyst%20Flex%20Surveys
78,I/O R.O. Enrolment Data Quality Assurance Officer,George Brown College,"Experience with data mining and reporting utilizing current software applications such as Microsoft Access, SQL data mining and reporting is required.","Toronto, ON",3 days ago,3,https://ca.indeed.com/jobs?q=I/O%20R.O.%20Enrolment%20Data%20Quality%20Assurance%20Officer%20George%20Brown%20College
20,Junior Data Analyst,FlightHub,Analyzing data sets and identifying trends and patterns; Our team will count on you to use data from various sources to identify issues as they happen and to…,"Montréal, QC",3 days ago,3,https://ca.indeed.com/jobs?q=Junior%20Data%20Analyst%20FlightHub
51,Analyste PMO junior - Junior PMO Analyst,Gameloft,"Being responsible for the regular data analytics, report generation and key performance indicators. 1 year of experience in project management or PMO-related…","Montréal, QC",3 days ago,3,https://ca.indeed.com/jobs?q=Analyste%20PMO%20junior%20-%20Junior%20PMO%20Analyst%20Gameloft
